In [2]:
from pyproj import CRS
import geopandas as gpd
import folium
import requests
import geojson

# URL to the WFS server
url = 'https://paituli.csc.fi/geoserver/paituli/wfs?'

# Specify parameters (read data in json format). 
params = dict(service='wfs', 
              version='2.0.0', 
              request='GetFeature', 
              typeName='paituli:tike_paavo_2021', 
              outputFormat='json')

# Fetch data from WFS using requests
r = requests.get(url, params=params)
    
# Create GeoDataFrame from geojson
data = gpd.GeoDataFrame.from_features(geojson.loads(r.content))

# Add description of crs from 3067 and reproject to Web Mercator     
data.crs = CRS.from_epsg(3067)
data = data.to_crs(3857)

In [3]:
# Use only needed columns
geo_series = data[['geometry','postinumer','nimi']]

# Create map instance
m = folium.Map(location = [64,27],
               zoom_start=5,
               tiles='CartoDBdark_matter',
               control_scale=True,
               max_bounds=True
              )
# Add features
folium.features.GeoJson(geo_series,
                        tooltip=folium.features.GeoJsonTooltip(fields=['postinumer','nimi'],
                                                                aliases=['Postal code', 'Name'],
                                                                labels=True,
                                                                sticky=True
                                                                            )
                       ).add_to(m)
    
folium.LayerControl().add_to(m)
m.save('docs/Postal_areas.html')